In [1]:
#import libraries
import numpy as np
from numpy import asarray
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# import gensim
import re
import joblib

2021-11-25 19:33:12.323950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-25 19:33:12.324000: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from os import listdir
dfs=[]
filepaths = ['../data/' + f for f in listdir('../data') if f.startswith('s') ]
for f in filepaths:
    df = pd.read_csv(f,encoding='utf-16')
    dfs.append(df)
df=pd.concat(dfs, ignore_index=True)

In [3]:
len(df)

37961

In [4]:
df.head(10)

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1
5,select * from users where id = '1' or @ @1 = ...,1
6,"select * from users where id = 1 or 1#"" ( un...",1
7,' select name from syscolumns where id = ( ...,1
8,select * from users where id = 1 +$+ or 1 = 1...,1
9,"1; ( load_file ( char ( 47,101,116,99,47,112,9...",1


In [5]:
#adding admin, user and password samples
password = pd.read_table('../data/10-million-password-list-top-1000000.txt')
password.dropna(inplace =True)


In [6]:
to_add = password[password['123456'].str.contains('user|pass|admin|password|adminstrator')].head(500)
to_add.rename(columns = {'123456':'Sentence'}, inplace = True)
to_add['Label'] = 0
to_add.head()

,Sentence,Label
0,password,0
73,pass,0
305,password1,0
409,passw0rd,0
751,passion,0


In [7]:
### cleaning
df = df[~df['Sentence'].isna()]

In [8]:
df2= pd.concat([df, to_add])
df2

,Sentence,Label
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1
5,select * from users where id = '1' or @ @1 = ...,1
...,...,...
203242,password1981,0
203243,password@123,0
203247,passWORD,0
203248,passwolf,0


In [9]:
df2.to_csv('../data/training.csv')

In [10]:

def clean(text):
    pattern = re.compile(r"\d*([^\d\W]+)\d*") # removes only those numbers appearing with words but leaves individual numbers alone
    
    return pattern.sub(r"\1", text)

In [11]:
vectorizer = CountVectorizer( min_df=2, max_df=0.7, stop_words=stopwords,token_pattern ='[a-zA-Z0-9$&+,:;=?@#|<>.^*()%!-]+')
X = vectorizer.fit_transform(df2['Sentence'].apply(lambda x:clean(x)))
y = df2['Label']
# X = vectorizer.fit_transform(df['words'])


In [12]:
# transformed_posts=pd.DataFrame(posts)

In [13]:
# posts

In [14]:
vectorizer.get_feature_names()[:10]

['!',
 '!)',
 '!<1',
 '!<1,version',
 '!<@',
 '!<@,version',
 '#',
 '#,@@version',
 '$',
 '$#']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
### base model logistic regression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy of logistic regression: %.3f "% accuracy_score(y_pred,y_test))
print("F1_score of logistic regression: %.3f "%f1_score(y_pred,y_test))
print("auc_roc of logistic regression: %.3f "%roc_auc_score(y_pred,y_test))

Accuracy of logistic regression: 0.993 
F1_score of logistic regression: 0.988 
auc_roc of logistic regression: 0.994 


In [17]:
### base model logistic regression
svc_clf = SVC(random_state=0).fit(X_train, y_train)
y_pred = svc_clf.predict(X_test)
print("Accuracy of SVC : %.3f "% accuracy_score(y_pred,y_test))
print("F1_score of SVC: %.3f "%f1_score(y_pred,y_test))
print("auc_roc of SVC: %.3f "%roc_auc_score(y_pred,y_test))

Accuracy of SVC : 0.993 
F1_score of SVC: 0.989 
auc_roc of SVC: 0.995 


In [18]:
dt = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = dt.predict(X_test)
print("Accuracy of Decision Tree Classifier : %.3f "% accuracy_score(y_pred,y_test))
print("F1_score of Decision Tree Classifier: %.3f "%f1_score(y_pred,y_test))
print("auc_roc of Decision Tree Classifier: %.3f "%roc_auc_score(y_pred,y_test))

Accuracy of Decision Tree Classifier : 0.997 
F1_score of Decision Tree Classifier: 0.995 
auc_roc of Decision Tree Classifier: 0.996 


In [19]:
# Create param grid.

param_grid = [
    
        {
     'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['liblinear']}
#     {'m' : [DecisionTreeClassifier()],
#      'pca__n_components':list(range(2,10)),
#     'm__max_depth' : list(range(2,10)),
#     'm__min_samples_leaf': [5, 10, 20, 50, 100],
#     'm__criterion': ["gini", "entropy"]
# }
#       {'m' : [GradientBoostingClassifier()],
#     'm__max_depth' : list(range(2,10)),
#       'm__n_estimators':list(range(100,500,50))}
    
        ]

# Create grid search object

clf = GridSearchCV(clf, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

In [20]:
clf.fit(X,y)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    7.0s finished


GridSearchCV(cv=5, estimator=LogisticRegression(random_state=0), n_jobs=-1,
             param_grid=[{'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                          'penalty': ['l1', 'l2'], 'solver': ['liblinear']}],
             verbose=True)

In [21]:
model = clf.best_estimator_

In [22]:
# save the model to disk
filename = '../model/finalized_model.sav'
joblib.dump(model, open(filename, 'wb'))

In [23]:
yhat = model.predict(X_test)

### END ###

In [24]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
input_dim = X.shape[1]  # Number of features
cvscores = []

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                287200    
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 1024)              11264     
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1025      
                                                        

2021-11-25 19:33:37.906067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-25 19:33:37.906108: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-25 19:33:37.906137: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2021-11-25 19:33:37.906402: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)

Epoch 1/10


2021-11-25 19:33:38.876231: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/home/bosen/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 273, in __call__
    return func(device, token, args)

  File "/home/bosen/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 151, in __call__
    outputs = self._call(device, args)

  File "/home/bosen/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 158, in _call
    ret = self._func(*args)

  File "/home/bosen/.local/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 649, in wrapper
    return func(*args, **kwargs)

  File "/home/bosen/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 476, in py_method
    return [slice_array(inp) for inp in flat_inputs]

  File "/home/bosen/.local/lib/python3.8/site-package

InvalidArgumentError:  TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/home/bosen/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 273, in __call__
    return func(device, token, args)

  File "/home/bosen/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 151, in __call__
    outputs = self._call(device, args)

  File "/home/bosen/.local/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 158, in _call
    ret = self._func(*args)

  File "/home/bosen/.local/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 649, in wrapper
    return func(*args, **kwargs)

  File "/home/bosen/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 476, in py_method
    return [slice_array(inp) for inp in flat_inputs]

  File "/home/bosen/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 476, in <listcomp>
    return [s

In [ ]:
# Save the model
filepath = './saved_model'
save_model(model, filepath)

In [ ]:
test_model = load_model(filepath, compile=True)
yhat = test_model.predict(X_test)

In [ ]:
print("Accuracy of Neural network: %.3f "%accuracy_score(np.rint(yhat),y_test))
print("F1_score of Neural network: %.3f "%f1_score(np.rint(yhat),y_test))
print("auc_roc of Neural network: %.3f "%roc_auc_score(np.rint(yhat),y_test))

In [ ]:
test = pd.read_csv('../data/Malicious_data.csv')
test_cv = vectorizer.transform(test.Payload[test['Classification']=='SQLI'])
pred= model.predict(test_cv)
np.rint(pred)

In [ ]:
yhat = test_model.predict(X_test)

In [ ]:

print("F1_score of neural network: %.3f "%f1_score(yhat,y_test))

In [ ]:
df[df['Sentence'].str.contains('Password')]

In [ ]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

In [ ]:
glove_vectors.most_similar('username')